In [1]:
from wordcloud import WordCloud, STOPWORDS
import urllib.request
from urllib.request import urlopen
import requests 
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import numpy as np
from datetime import datetime,timedelta
import copy
import hanja
from hanja import hangul  
import time
import random
from tqdm import tqdm

# 데이터 크롤링

In [2]:
while(1):
    start_date=input('데이터를 모을 시작날짜를 입력하세요 ex) 2019-01-01: ')
    if (start_date[:4].isdecimal()==False) | (start_date[5:7].isdecimal()==False) | (start_date[-2:].isdecimal()==False):
        print('다시 시작날짜를 입력하세요')
    elif (len(start_date)!=10):
        print('예시양식에 맞춰 다시 시작날짜를 입력하세요')
    else:
        break

while(1):
    end_date=input('데이터를 모을 마지막 날짜를 입력하세요(입력날짜 전날까지 크롤링됌.) ex) 2020-01-01:')
    if (end_date[:4].isdecimal()==False) | (end_date[5:7].isdecimal()==False) | (end_date[-2:].isdecimal()==False):
        print('다시 시작날짜를 입력하세요')
        
    elif (len(start_date)!=10):
        print('예시양식에 맞춰 다시 시작날짜를 입력하세요')
        
    else:
        break

        

데이터를 모을 시작날짜를 입력하세요 ex) 2019-01-01: 2020-10-31
데이터를 모을 마지막 날짜를 입력하세요(입력날짜 전날까지 크롤링됌.) ex) 2020-01-01:2020-11-04


In [3]:

start_date=start_date.split('-')
end_date=end_date.split('-')

start_year=start_date[0]
start_month=start_date[1]
start_day=start_date[2]

end_year=end_date[0]
end_month=end_date[1]
end_day=end_date[2]


In [4]:
# krx API -> kospi데이터 수집          
from pykrx import stock                        # pip instasll pykrx
start_date__=''.join(start_date)
end_date__=''.join(end_date)
kospi_ = stock.get_index_ohlcv_by_date(start_date__, end_date__, "1001")
kospi_.columns = ['Open','High','Low','Close','Volume']
kospi_

,Open,High,Low,Close,Volume
날짜,,,,,
2020-11-02,2276.61,2302.64,2267.95,2300.16,678501000
2020-11-03,2315.81,2344.77,2315.30,2343.31,887445000
2020-11-04,2361.32,2371.14,2339.95,2357.32,986786000


In [5]:
lst=[]
for i in range(len(kospi_['Close'])-1):
    a=(kospi_.iloc[i+1,3])
    b=kospi_.iloc[i,3]
    lst.append(round(a-b,2))
    
lst.append(lst[-1])
print(len(lst))
print(len(kospi_))

3
3


In [6]:
kospi_['등락폭']=lst
kospi_

,Open,High,Low,Close,Volume,등락폭
날짜,,,,,,
2020-11-02,2276.61,2302.64,2267.95,2300.16,678501000,43.15
2020-11-03,2315.81,2344.77,2315.30,2343.31,887445000,14.01
2020-11-04,2361.32,2371.14,2339.95,2357.32,986786000,14.01


In [7]:
# 한경 경제부문 기사 크롤링 (오후 3시까지의 기사만 크롤링)
from tqdm import notebook

dom='economy'

kospi_dates=kospi_.index
list_economy=[]
for i in  tqdm(kospi_dates,desc='iterate range 100'):
    d=i
    #d=d-timedelta(days=1)             # 코스피날짜 당일날 기사를 크롤링
    d=datetime.strftime(d,'%Y-%m-%d')
    dat=d
    d=d.split('-')
    year=d[0]
    month=d[1]
    day=d[2]
    
    flag=True
    list2=[dat]
    for j in range(100) : # range : 페이지 수 입력
    
        page = str(j+1)
        page2=copy.deepcopy(page)
        
        url = f'https://hankyung.com/{dom}?date={year}.{month}.{day}&page={page}'
        #url = 'https://www.hankyung.com/economy?date=2020.04.01'
        response = requests.get(url)
        html = response.text
        time.sleep(random.uniform(2,4)) 
        soup = BeautifulSoup(response.content.decode('UTF-8','replace'),'lxml')
        
        tit2 = soup.find_all('h3', 'tit') # 나머지 
        tit3=soup.find_all('span','time') # 시간대
        
        if tit2==[]:
            break
        
        for i,j in zip(range(len(tit2)),range(len(tit3))):
            b=str(tit3[j])
            
            b2=b.split('>')[1]
            b3=b2[-12:-6]
            h=b3[:3]
            m=b3[4:]
            
            if int(h)<15:
                #print(int(h))
                a=str(tit2[i])
                a2 = a.split('>')[2]
                a3 = a2.split('<')[0]
                list2.append(a3)
                if len(list2)>50:
                    flag=False
                    break
                
            else:
                continue
                
        if flag==False:
            break
            
    list_economy.append(list2)

iterate range 100: 100%|█████████████████████████████████████████████████████████████████| 3/3 [01:41<00:00, 33.83s/it]


In [8]:
pd_list1=pd.DataFrame(list_economy)
pd_list1[:5]

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2020-11-02,에스티팜 3분기 영업손실 78억원…적자 확대,"홈서비스 미소, 대우건설과 홈클리닝 업무협약 ""서비스 제휴 기업 확대할 것""","신성금고, 은행 맞춤형 대여금고 서비스 '볼트1932' 출시","중진공, 케이지에듀원과 성과공유제 계약",'윌리엄스 소노마' 출점 전략 바꾼 현대리바트,"홈앤쇼핑도 '코세페'…11월 페이백, 초특가판매 등 이벤트 풍성","강남제비스코, 내부 판넬용 원 코팅 도료 'KCP860' 개발","엔젤투자협회, '스타트업 채용 페스티벌' 13일까지 진행","""아기상어 뚜루루뚜루~"" 핑크퐁아기상어 유튜브 조회수 1위 등극",...,"벤츠 ""'더 뉴 EQC' 전기차 보조금 11월 말까지 연장""",작년분 근로·자녀장려금 기한 놓친 가구 12월 1일까지 신청가능,풍력발전 인증시험 기술 개발…비용은 해외의 절반,공정위 '데이터룸' 만들어 자료열람 등 기업 방어권 보장,"가상자산 사업자, 선불카드·모바일 상품권 취급 못한다",내년부터 금융사 1회 방문이면 모든 퇴직연금 '한번에' 이동,"금융위 ""가상자산 사업자, FIU 신고해야…제도화는 아냐""",서울시 내년 예산 40조원 넘었다…5120억원 증액,루크 동커볼케의 귀환…현대차그룹 CCO 맡는다,현대차그룹 '브랜드 경쟁력 강화' CCO 신설…루크 동커볼케 복귀
1,2020-11-03,"홍남기 ""중저가 주택 재산세 경감 오후 4시 발표""",중국 전기차 강력 드라이브…2025년 판매 20% 친환경차로,"[속보] 홍남기 부총리 사의 표명…""대주주 확대 유예에 책임""","[2보] 홍남기 ""주식 양도세 대주주 기준 10억원 유지""","[속보] 홍남기 ""중저가 주택 재산세 경감 오후 4시 발표""",팬오션 3분기 영업이익 629억원…작년 동기 대비 0.8%↓,"[속보] 홍남기 ""중저가 주택 재산세 경감 오후 4시 발표""","[속보] 홍남기 부총리, 사직서 제출…문 대통령 '반려'","[속보] 홍남기 ""대주주 기준 10억원 유지""",...,"더벤티, '슬기로운 더벤티 생활' 론칭해","""누적된 청년실업…2021년 상반기 채용시장 최악될 것""",대구경북 10월 소비자물가 9월보다 소폭 하락,"코로나에 비대면 결제 17% 늘고, 실물카드 결제 6% 줄어","한불모터스, 푸조 3008∙5008에 7개월 할부 면제","""엄마, 나 급전 필요해""…'카톡 피싱' 급증","공정위, 가맹법 위반한 '이화수전통육개장'에 시정명령","보험업, 금융업권 '민원왕'…한 해 5만여건","제주도관광협회, '세계관광의 날' 기념식…마스크 착용 홍보활동",[하이빔]자동차 레몬법 낮잠에 블랙컨슈머도 논란
2,2020-11-04,트럼프 재선 가능성에 친환경주 급락…대형기술주↑,'트위지 레이스'로 기부 독려 나선 르노삼성,"자영업자 10명 중 2명 ""취업 하다하다 안돼서 창업""","전북 지자체-기관·기업, 수소경제 생태계 조성 '박차'","""코로나일상 시대…방역·보건·경제·사회 포괄관점 필요""","검찰, 라임사태 관련 '우리은행' 압수수색",9월 온라인쇼핑 14.7조 3개월 연속 '사상 최대'…배달 91%↑,부산 마이스 로드쇼 하반기 행사 원도심에서 열려,미래차 대중화 시대 본격 지원…환경부 '미래차 추진단' 구성,...,"베트남 특별입국 기업들 ""대체로 만족…격리기간은 단축해야""",고가건물 산 연예인·20억대 골프빌리지 산 법인 등 세무조사,푸드트럭·주차장 등 국유재산 소상공인에 우선 임대한다,빅데이터로 빌라 시세·담보가치 산정…지정대리인 2곳 지정,"""회계이슈 사전예고제 안착""…기업들 오류 수정 빨라졌다",'회사차' 슈퍼카 굴린 사주…가족기획사로 탈세한 유명 연예인,코로나 직격탄 맞은 4050 자영업자…1년새 24만명 줄어,246만명 '쉬었음'…1년 전보다 29만명 증가,가족 명의로 기획사 세워 탈세한 유명 연예인 딱 걸렸다,"15년간 이대생의 식탁 '닥터로빈', 배달로 제2의 승부수"


In [9]:
# 각 열에 null값 개수 확인
pd_list1.isnull().sum()    

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
dtype: int64

In [10]:
print(list(pd_list1.columns))  # 0 : 날짜 / 1 ~50 : 기사제목

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [11]:
# pd_list1.to_csv('./pd_list1_hourtest.csv',encoding='utf-8-sig')

In [12]:
# 한경신문 국제부문 기사 크롤링 (오후 3시까지의 기사만 크롤링)

dom='international'

kospi_dates=kospi_.index
list_international=[]
for i in tqdm(kospi_dates):
    d=i
    #d=d-timedelta(days=1)   # 코스피날짜 당일날 기사를 크롤링
    d=datetime.strftime(d,'%Y-%m-%d')
    dat=d
    d=d.split('-')
    year=d[0]
    month=d[1]
    day=d[2]
    
    flag=True
    list2=[dat]
    for j in range(100) : # range : 페이지 수 입력
    
        page = str(j+1)
        page2=copy.deepcopy(page)
        
        url = f'https://hankyung.com/{dom}?date={year}.{month}.{day}&page={page}'
        response = requests.get(url)
        html = response.text
        time.sleep(random.uniform(2,4)) 
        soup = BeautifulSoup(response.content.decode('UTF-8','replace'),'lxml')
        tit2 = soup.find_all('h3', 'tit') # 나머지 
        tit3=soup.find_all('span','time') # 시간대 
        
        if tit2==[]:
            break

        
        for i,j in zip(range(len(tit2)),range(len(tit3))):
            b=str(tit3[j])
            
            b2=b.split('>')[1]
            b3=b2[-12:-6]
            h=b3[:3]
            m=b3[4:]
            
            if int(h)<15:
                a=str(tit2[i])
                a2 = a.split('>')[2]
                a3 = a2.split('<')[0]
                list2.append(a3)
                if len(list2)>50:
                    flag=False
                    break
                
            else:
                continue
                
        if flag==False:
            break
            
    list_international.append(list2)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:22<00:00, 27.52s/it]


In [13]:
pd_list2=pd.DataFrame(list_international)

In [14]:
pd_list2.isnull().sum()         # 각 열의 Null값 확인

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
dtype: int64

In [15]:
print(pd_list1.shape)  # 날짜, 기사들(20개) -> 열이 21개
print(pd_list2.shape)  # 기사들(20개)       -> 열이 20개

(3, 51)
(3, 51)


In [16]:
# pd_list2.to_csv('./inter_temp2.csv',encoding='utf-8-sig')

In [17]:
pd_list1=pd_list1.rename(columns={0:'Date'})  
pd_list2=pd_list2.rename(columns={0:'Date'})

In [18]:
# 경제면 기사(pd_list1)와 국제면 기사(pd_list2)를 같은 날짜를 기준으로 합치는 과정
pd_list=pd.merge(pd_list1,pd_list2,on='Date')

In [19]:
len(pd_list.columns)  # 날짜 : 1개, 기사 : 40개

101

In [20]:
print(len(kospi_['등락폭']),len(pd_list))  # 두값이 같아야 아래 과정이 올바르게 진행됩니다.

3 3


In [21]:
print(pd_list1.shape)
print(kospi_.shape)
pd_list['label_val']=list(kospi_['등락폭'])

# 기준 : 0
lst=[]
for i in kospi_['등락폭']:
    if i<0:
        lst.append(0)        # 등락폭이 0미만 -> 0로 라벨링
    
    elif i>0:
        lst.append(1)        # 등락폭이 0초과 -> 1로 라벨링
    else:
        lst.append(2)        # 등락폭이 0 -> 2로 라벨링
pd_list['label']=lst
pd_list[:5]

(3, 51)
(3, 6)


,Date,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,43_y,44_y,45_y,46_y,47_y,48_y,49_y,50_y,label_val,label
0,2020-11-02,에스티팜 3분기 영업손실 78억원…적자 확대,"홈서비스 미소, 대우건설과 홈클리닝 업무협약 ""서비스 제휴 기업 확대할 것""","신성금고, 은행 맞춤형 대여금고 서비스 '볼트1932' 출시","중진공, 케이지에듀원과 성과공유제 계약",'윌리엄스 소노마' 출점 전략 바꾼 현대리바트,"홈앤쇼핑도 '코세페'…11월 페이백, 초특가판매 등 이벤트 풍성","강남제비스코, 내부 판넬용 원 코팅 도료 'KCP860' 개발","엔젤투자협회, '스타트업 채용 페스티벌' 13일까지 진행","""아기상어 뚜루루뚜루~"" 핑크퐁아기상어 유튜브 조회수 1위 등극",...,"""총 들고 투표하겠다""…미 대선 앞두고 긴장감 도는 투표소",터키·그리스 강진 사망자 75명으로 늘어…부상자 1천명 육박,"美매체 ""트럼프, 대선 당일 조기 승리 선언""…대혼란 가능성","숀 코너리 유족 ""코너리, 말년에 치매 앓아""",눈덩이처럼 커지는 서학개미 환차손…원‧달러 환율 향방은 [한상춘 칼럼],미국 대선 속 '마이웨이'…중국 내수 강화·우군 확보 총력전,바이든 또 말실수? 알고보니 가짜영상,"중국, 10년 단위 인구센서스 개시…""14억2천만 명 예상""",43.15,1
1,2020-11-03,"홍남기 ""중저가 주택 재산세 경감 오후 4시 발표""",중국 전기차 강력 드라이브…2025년 판매 20% 친환경차로,"[속보] 홍남기 부총리 사의 표명…""대주주 확대 유예에 책임""","[2보] 홍남기 ""주식 양도세 대주주 기준 10억원 유지""","[속보] 홍남기 ""중저가 주택 재산세 경감 오후 4시 발표""",팬오션 3분기 영업이익 629억원…작년 동기 대비 0.8%↓,"[속보] 홍남기 ""중저가 주택 재산세 경감 오후 4시 발표""","[속보] 홍남기 부총리, 사직서 제출…문 대통령 '반려'","[속보] 홍남기 ""대주주 기준 10억원 유지""",...,백악관에 등장한 '트럼프 장벽'…美 '일촉즉발' [2020 미국의 선택],"폼페이오, 홍콩 범민주파 의원 체포에 ""법집행 남용"" 규탄",美 대선 승자 바로 확정 못하면 경제에 큰 '타격' 온다 [2020 미국의 선택],"중국, 미 '중국위협론'에 ""미국이 진정한 '타노스'""",SEC 올해 월가 제재금 5조원 넘어…역대 최대,"WP ""미 대선 뒤 '통치 공백' 두달 심각한 경제위기 우려""","""'바이든-중국 연계설' 뒤엔 中 반체제 재벌 궈원구이""","태국국왕 '타협' 발언, 3개월 시위사태 해법논의 촉진할까",14.01,1
2,2020-11-04,트럼프 재선 가능성에 친환경주 급락…대형기술주↑,'트위지 레이스'로 기부 독려 나선 르노삼성,"자영업자 10명 중 2명 ""취업 하다하다 안돼서 창업""","전북 지자체-기관·기업, 수소경제 생태계 조성 '박차'","""코로나일상 시대…방역·보건·경제·사회 포괄관점 필요""","검찰, 라임사태 관련 '우리은행' 압수수색",9월 온라인쇼핑 14.7조 3개월 연속 '사상 최대'…배달 91%↑,부산 마이스 로드쇼 하반기 행사 원도심에서 열려,미래차 대중화 시대 본격 지원…환경부 '미래차 추진단' 구성,...,사전투표만 1억명 '역대 최고'…우편투표가 당락 가를 듯,"[미 대선] 극우 음모론 '큐어넌' 신봉자, 하원의원 당선","트럼프, 전체 득표수 342만표 우위…오후 1시 기준 [미국 대선]","[미 대선] 트럼프, 전체 득표수에서 342만표 우위…오후 1시 현재","[속보] ""트럼프, 플로리다 승리""…폭스뉴스",[속보] 선거인단 확보 트럼프 118명 vs 바이든 209명,[속보] '승부처' 노스캐롤라이나 득표율…트럼프 50.1% 바이든 48.7%,"[속보] 폭스뉴스 ""트럼프 핵심 경합주 플로리다서 승리"" 예측",14.01,1


In [22]:
pd_list.shape

(3, 103)

In [23]:
date=[]
title=[]
label_val=[]
label=[]
for i in range(len(pd_list)):
    for j in range(1,pd_list.shape[1]-2):
        date.append(pd_list.iloc[i,0])
        title.append(pd_list.iloc[i,j])
        label_val.append(pd_list.iloc[i,-2])   # 코스피 등락폭 
        label.append(pd_list.iloc[i,-1])       # 코스피 등락폭에 따른 라벨링(음수:0,양수:1,0:2)

In [24]:
news=pd.DataFrame({'날짜':date,'기사':title,'label_val':label_val,'label':label})

In [25]:
print(news.isnull().sum())  

날짜           0
기사           0
label_val    0
label        0
dtype: int64


In [26]:
news=news.dropna()  # 기사가 없는 경우(널값), 행삭제 처리

In [27]:
# news.to_csv('./news.csv',encoding='utf-8-sig',index=False)

In [28]:
news

,날짜,기사,label_val,label
0,2020-11-02,에스티팜 3분기 영업손실 78억원…적자 확대,43.15,1
1,2020-11-02,"홈서비스 미소, 대우건설과 홈클리닝 업무협약 ""서비스 제휴 기업 확대할 것""",43.15,1
2,2020-11-02,"신성금고, 은행 맞춤형 대여금고 서비스 '볼트1932' 출시",43.15,1
3,2020-11-02,"중진공, 케이지에듀원과 성과공유제 계약",43.15,1
4,2020-11-02,'윌리엄스 소노마' 출점 전략 바꾼 현대리바트,43.15,1
...,...,...,...,...
295,2020-11-04,"[미 대선] 트럼프, 전체 득표수에서 342만표 우위…오후 1시 현재",14.01,1
296,2020-11-04,"[속보] ""트럼프, 플로리다 승리""…폭스뉴스",14.01,1
297,2020-11-04,[속보] 선거인단 확보 트럼프 118명 vs 바이든 209명,14.01,1
298,2020-11-04,[속보] '승부처' 노스캐롤라이나 득표율…트럼프 50.1% 바이든 48.7%,14.01,1


In [29]:
# # 19~20년도 한경 뉴스 통합본 
# news=pd.read_excel('C:/Users/Kim SeoJeong/Turn_Around_PJ/Project_TA/19_20_HK_News/HKNews_1920.xlsx')
# news=news.dropna()  # 기사가 없는 경우(널값), 행삭제 처리

In [30]:
docs=news['기사'].to_list()
docs[0]

'에스티팜 3분기 영업손실 78억원…적자 확대'

# 데이터 전처리

In [31]:
import re
clean_docs = []
pattern = '[-=+,#/\?:·;^$.%@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'   # 특수문자 제거
pattern2='[]'
cnt=0
news_remov_index=[]
for i in range(len(docs)) :
    sen=docs[i]
    print(sen)
    span=[]
    for p in range(len(docs[i])):
        if docs[i][p]=='[':
            span.append(p)
        elif docs[i][p]==']':
            span.append(p)
        else:
            continue
    if len(span)==2:
        sen=sen[0:span[0]]+sen[span[1]+1::]     # 칼럼제목([]로 둘러싸인 문자열)을 제거
    
    sen = re.sub(pattern,"",sen) # pattern에서 정의한 특수문자 정규표현식 을 제거 
    sen = re.sub('[0-9]+','',sen)
    #sen = re.sub('[一-龥]','',sen)
    #sen = re.sub('[ㄱ-ㅣ가-힣]+','',sen)
    #sen = re.sub('[a-zA-Z]','',sen)
    sen = sen.strip() # 양옆의 공백을 제거 
    
    #news['기사'][i]=sen
    clean_docs.append(sen)

에스티팜 3분기 영업손실 78억원…적자 확대
홈서비스 미소, 대우건설과 홈클리닝 업무협약 "서비스 제휴 기업 확대할 것"
신성금고, 은행 맞춤형 대여금고 서비스 '볼트1932' 출시
중진공, 케이지에듀원과 성과공유제 계약
'윌리엄스 소노마' 출점 전략 바꾼 현대리바트
홈앤쇼핑도 '코세페'…11월 페이백, 초특가판매 등 이벤트 풍성
강남제비스코, 내부 판넬용 원 코팅 도료 'KCP860' 개발
엔젤투자협회, '스타트업 채용 페스티벌' 13일까지 진행
"아기상어 뚜루루뚜루~" 핑크퐁아기상어 유튜브 조회수 1위 등극
진에어, 국내 LCC 첫 美 본토 운항…여객기로 화물 날랐다
재영솔루텍 "베트남 자회사에 113억원 출자"
원화 초강세에…올 1인당 국민소득 3만달러 유지 [김익환의 외환·금융 워치]
군장에너지·삼광글라스·이테크건설 합병 'SGC에너지' 공식출범
'교촌치킨' 교촌에프앤비 공모가 1만2천300원…3∼4일 일반청약
'라이언킹' 이동국 은퇴식 직접 간 정의선…"연락합시다"
현대硏 "바이든 당선 때 韓 성장률 최대 0.4%포인트 오를 것"
LG전자, 美서 '에너지스타 데이' 맞아 고효율 가전 기부
국내 최대 콘텐츠 전시회 '광주 에이스 페어' 5일 개막
"어떻게 하루아침에…" 샤넬백 사러 간 예비신부 '당황'
동시 가입 혜택이 팡팡…KB국민은행, 금융·통신 결합 상품 출시
화성시, 도내 첫 버스공영제 시행…3일부터 2개 노선 운행
공정위, 한국형 '데이터룸' 만든다…"기업 방어권 보장"
미국이냐 중국이냐…산업硏 "대선 누가 되든 韓입장 정리해야"
서울 관악구 낙성대동에 '강감찬 도시농업센터' 건립
음성군, 수소 기반 에너지산업 핵심도시로 거듭난다
웰컴저축, 특수신발 만들어 의족 교사 KLPGA 준회원 선발 돕는다
가상자산 범위에서 선불카드·모바일 상품권은 제외
초특가 할인에 지갑 열었다…신세계 '쓱데이' 매출 6400억
현대해상·현대기아차, '모빌리티 스타트업' 공동육성 나선다
건산연 "내년 전국 집값 0.5% 하락…전셋값 5.0% 상승 전망"
내년부터 퇴직연

In [32]:
clean_docs[:10]

['에스티팜 분기 영업손실 억원적자 확대',
 '홈서비스 미소 대우건설과 홈클리닝 업무협약 서비스 제휴 기업 확대할 것',
 '신성금고 은행 맞춤형 대여금고 서비스 볼트 출시',
 '중진공 케이지에듀원과 성과공유제 계약',
 '윌리엄스 소노마 출점 전략 바꾼 현대리바트',
 '홈앤쇼핑도 코세페월 페이백 초특가판매 등 이벤트 풍성',
 '강남제비스코 내부 판넬용 원 코팅 도료 KCP 개발',
 '엔젤투자협회 스타트업 채용 페스티벌 일까지 진행',
 '아기상어 뚜루루뚜루 핑크퐁아기상어 유튜브 조회수 위 등극',
 '진에어 국내 LCC 첫 美 본토 운항여객기로 화물 날랐다']

In [33]:
# 위과정으로 정제된 문장을 news데이터프레임에 넣어주는 과정 (nan값 제거)
for i in range(len(clean_docs)):
    if clean_docs[i]=='':
        print(i)
    
for i in range(len(news['기사'])):
    if clean_docs[i]=='':
        news['기사']==float('nan')
        continue
        
    news['기사']=clean_docs[i]
    
news=news.dropna()

In [34]:
print(news.shape[1])             # 위,아래 값이 같아야 함
print(len(clean_docs))

4
300


Mecab적용하기 (조사, 접미사, 부사, 어말어미는 제거하여 불용어를 처리)

In [35]:
#from konlpy.tag import Okt       # 형태소분석기중, Mecab를 사용 
from tqdm import tqdm
from konlpy.tag import Mecab
tagger=Mecab('C:/mecab/mecab-ko-dic')

In [36]:
josa=['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC']                    # 조사
jeop_misa_and_doosa=['XSN','XSV','XSA','XPN']                                 # 접미사
boosa=['MAG','MAJ']                                                           # 부사
eomaleomy=['EF','EC','ETN','ETM','EP']                                        # 어말어미   (EP는 선어말어미)


sentence=[]
for doc in tqdm(clean_docs):
    if doc=='':
        sentence.append(['Nan'])
        continue
        
    p = tagger.pos(doc)
    tokens=[]
    
    for j in p:
        i=j[1]
        if (i in josa) | (i in jeop_misa_and_doosa) | (i in boosa) | (i in eomaleomy):
            continue
        tokens.append(j[0])
        
   
    sentence.append(tokens)

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 318.29it/s]


In [37]:
sentence_copy=copy.deepcopy(sentence)        
sentence_copy[0]
print(len(sentence_copy))
print(news.shape)

300
(300, 4)


In [38]:
# 나라한자를 나라한글로 Replace하는 과정
# ex) [美 -> 미국], [日 -> 일본] 등 
trans_cntry={'美':'미국','佛':'프랑스','日':'일본','中': '중국','露': '러시아','獨':'독일','北':'북한','英':'영국','弗':'달러' }

for i in range(len(sentence_copy)):
    for j in range(len(sentence_copy[i])):
        if sentence_copy[i][j] in trans_cntry.keys():
            sentence_copy[i][j]=trans_cntry[sentence_copy[i][j]]
            
sentence=copy.deepcopy(sentence_copy)

In [39]:
# 구축해놨던 Top1000단어들의 도메인문서를 기반으로 기사내 단어가 도메인으로 Replace되는 과정
# ex) 오바마 -> 미국대통령
ref_lst=pd.read_excel('C:/Users/Kim SeoJeong/Turn_Around_PJ/Project_TA/TF1000개의단어사전과Mecab적용하기/단어사전_final.xlsx',sheet_name='Conditions')
ref_lst

for i in tqdm(range(len(sentence))):
    for j in range(len(sentence[i])):
        for a in ref_lst.columns:
            if sentence[i][j] in list(ref_lst[a]):                
                sentence[i][j]=a
                break

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:02<00:00, 118.84it/s]


In [40]:
# 한자 -> 한글로 변환되는 과정
# ex) 南北 -> 남북 

import hanja
from hanja import hangul

sen=[]
#a=hanja.translate('北美','substitution')
for i in sentence:
    sub=[]
    for j in i:
        j=hanja.translate(j,'substitution')
        sub.append(j)
    sen.append(sub)
sentence=copy.deepcopy(sen)

In [41]:
# 각 나라줄임말 사전만드는 과정
# 기사내의 [미중] [미러] -> [미국, 중국], [미국, 러시아]로 변환해주기 위함

from itertools import combinations
from itertools import permutations

item=['한','중','일','북','미','러']
d_country={'한':'한국','중':'중국','일':'일본','북':'북한','미':'미국','러':'러시아'}
main_country3=list(map(''.join,permutations(item,3)))
main_country2=list(map(''.join,permutations(item,2)))
#print(main_country3)

dic_country={}                         # '한중일':'한국,중국,일본'

# 3개짜리
for i in main_country3:
    if (i[1]=='한') | (i[2]=='한'):
        continue
    
    sub=[]
    for j in i:
        sub.append(d_country[j])
    
    dic_country[i]=sub
    
# 2개짜리
for i in main_country2:
    if (i[1]=='한'):
        continue
    
    sub=[]
    for j in i:
        sub.append(d_country[j])
    
    dic_country[i]=sub
     
dic_country['남북']=['남한','북한']

In [42]:
# 만든 나라줄임말 사전을 적용하여 Replace
# ex) 한중일 -> 한국,중국,일본 / 한중러 -> 한국,중국,러시아 / 한중미 -> 한국,중국,미국
import itertools

def flat_list(array): 
    a=[]
    for i in array:
        if type(i) == type(list()):
            a+=(flat_list(i))       
        else:
            a.append(i)
    return a

for i in range(len(sentence)):
    for j in range(len(sentence[i])):
        if sentence[i][j] in dic_country.keys():
            sentence[i][j]=dic_country[sentence[i][j]]
            
        else:
            continue
        sentence[i]=flat_list(sentence[i])
sentence[:10]

[['에스티', '팜', '분기', '영업', '손실', '억', '원', '적자', '확대'],
 ['홈서비스',
  '미소',
  '대우건설',
  '홈',
  '클리닝',
  '업무',
  '협약',
  '서비스',
  '제휴',
  '기업',
  '확대',
  '할',
  '것'],
 ['신성', '금고', '은행', '맞춤', '대여금', '서비스', '볼트', '출시'],
 ['한국공기업', '케이지', '에듀', '원', '성과', '공유제', '계약'],
 ['윌리엄스', '소노마', '출점', '전략', '바꾼', '한국기업', '리바트'],
 ['홈', '앤', '쇼핑', '코세', '페월', '페이', '백', '초', '특가', '판매', '등', '이벤트', '풍성'],
 ['강남', '비스코', '내부', '판넬', '원', '코팅', '도료', 'KCP', '개발'],
 ['엔젤', '투자', '협회', '스타트업', '채용', '페스티벌', '일', '진행'],
 ['아기',
  '상어',
  '뚜',
  '루루',
  '뚜',
  '핑크',
  '퐁',
  '아기',
  '상어',
  '유튜브',
  '조',
  '회수',
  '위',
  '등',
  '극'],
 ['진에어', '한국', 'LCC', '첫', '미국', '본토', '운항', '여객기', '화물', '날랐']]

In [43]:
# 각 리스트내에 한글자 빼기
sen=[]
for i in sentence:
    sub=[]
    for j in i:
        if len(j)>1:
            sub.append(j)
        else:
            continue
    sen.append(sub)

In [44]:
print(len(sen))
print(len(news['기사']))

300
300


In [45]:
print(len(news['label_val']),len(news['label']),len(news['날짜']),len(sen)) 

300 300 300 300


In [46]:
if (len(news['label_val'])==len(news['label'])==len(news['날짜'])==len(sen)):
    df=pd.DataFrame()
    df['label_val']=news['label_val']
    df['label']=news['label']
    df['Date']=news['날짜']
    df['title']=sen

else:
    print('Please Kernel Restart & Run ALL')

In [47]:
df=df[['Date','title','label_val','label']]

In [48]:
df.isnull().sum()

Date         0
title        0
label_val    0
label        0
dtype: int64

In [49]:
# 전처리후
df['title']

0                           [에스티, 분기, 영업, 손실, 적자, 확대]
1      [홈서비스, 미소, 대우건설, 클리닝, 업무, 협약, 서비스, 제휴, 기업, 확대]
2                  [신성, 금고, 은행, 맞춤, 대여금, 서비스, 볼트, 출시]
3                       [한국공기업, 케이지, 에듀, 성과, 공유제, 계약]
4                  [윌리엄스, 소노마, 출점, 전략, 바꾼, 한국기업, 리바트]
                            ...                      
295                      [미국대통령, 전체, 득표수, 우위, 오후, 현재]
296                        [미국대통령, 플로리, 승리, 외국기업, 뉴스]
297                       [선거인단, 확보, 미국대통령, vs, 미국인물]
298                  [승부처, 노스캐롤라이나, 득표율, 미국대통령, 미국인물]
299          [외국기업, 뉴스, 미국대통령, 핵심, 경합주, 플로리다, 승리, 예측]
Name: title, Length: 300, dtype: object

In [50]:
df

,Date,title,label_val,label
0,2020-11-02,"[에스티, 분기, 영업, 손실, 적자, 확대]",43.15,1
1,2020-11-02,"[홈서비스, 미소, 대우건설, 클리닝, 업무, 협약, 서비스, 제휴, 기업, 확대]",43.15,1
2,2020-11-02,"[신성, 금고, 은행, 맞춤, 대여금, 서비스, 볼트, 출시]",43.15,1
3,2020-11-02,"[한국공기업, 케이지, 에듀, 성과, 공유제, 계약]",43.15,1
4,2020-11-02,"[윌리엄스, 소노마, 출점, 전략, 바꾼, 한국기업, 리바트]",43.15,1
...,...,...,...,...
295,2020-11-04,"[미국대통령, 전체, 득표수, 우위, 오후, 현재]",14.01,1
296,2020-11-04,"[미국대통령, 플로리, 승리, 외국기업, 뉴스]",14.01,1
297,2020-11-04,"[선거인단, 확보, 미국대통령, vs, 미국인물]",14.01,1
298,2020-11-04,"[승부처, 노스캐롤라이나, 득표율, 미국대통령, 미국인물]",14.01,1


In [53]:
df.shape

(300, 4)

In [54]:
df

,Date,title,label_val,label
0,2020-11-02,"[에스티, 분기, 영업, 손실, 적자, 확대]",43.15,1
1,2020-11-02,"[홈서비스, 미소, 대우건설, 클리닝, 업무, 협약, 서비스, 제휴, 기업, 확대]",43.15,1
2,2020-11-02,"[신성, 금고, 은행, 맞춤, 대여금, 서비스, 볼트, 출시]",43.15,1
3,2020-11-02,"[한국공기업, 케이지, 에듀, 성과, 공유제, 계약]",43.15,1
4,2020-11-02,"[윌리엄스, 소노마, 출점, 전략, 바꾼, 한국기업, 리바트]",43.15,1
...,...,...,...,...
295,2020-11-04,"[미국대통령, 전체, 득표수, 우위, 오후, 현재]",14.01,1
296,2020-11-04,"[미국대통령, 플로리, 승리, 외국기업, 뉴스]",14.01,1
297,2020-11-04,"[선거인단, 확보, 미국대통령, vs, 미국인물]",14.01,1
298,2020-11-04,"[승부처, 노스캐롤라이나, 득표율, 미국대통령, 미국인물]",14.01,1


In [51]:
# df.to_csv('./AI_학습데이터_1920.csv',encoding='utf-8-sig',index=False)

# BERT 모델 적용

In [52]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing
from keras.layers import TimeDistributed as td
from keras.layers import Multiply
import os

Using TensorFlow backend.
